In [1]:
def evaluate_for_key(key):
    resultData = pd.read_csv('results/' + key + '.csv')     
    bugs = resultData.bug.unique()
    print("Project " + key)
    print(len(bugs))
    count1 = 0;
    count5 = 0;
    count10 = 0;
    mapValue = 0;
    mrr = 0;
    for bug in bugs:
        subData = resultData[resultData['bug'] == bug]
        subDataSorted = subData.sort_values('Result2',ascending=False)
        
        if subDataSorted.iloc[0,-2] == 1:
            count1 += 1;
        if 1 in subDataSorted.values[0:5,-2]:
            count5 += 1;
        if 1 in subDataSorted.values[0:10,-2]:
            count10 += 1;
        
        summe = 0;
        rr = 0;
        retrieved_d = 0;
        i = 0
        c = 0
        for resultV in subDataSorted.values[:,-2]:
            if resultV == 1:
                c += 1
                retrieved_d += 1;
                summe += retrieved_d /(i+1)
            if resultV == 1 and rr == 0:    
                rr += (1/ (i+1))
            i += 1
        if c != 0:
            mapValue += (summe/c)
        else: 
            print("c zero!")
            mapValue +=0
            #rr = 1
        mrr += rr
    return count1/len(bugs),count5/len(bugs),count10/len(bugs),mapValue/len(bugs),mrr/len(bugs)

In [2]:
import os
import pandas as pd

data_mp = pd.read_csv('statistic.csv')


data = pd.DataFrame()
for file in os.listdir("amalgam_olddata"):
    result = file.replace('smartshark_','').replace('_output.txt','').split("_")
    project = result[1]
    area = result[0]
    version = "_".join(result[3:])
    element = data_mp.loc[(data_mp['Project'] == area + "_" + project) & (data_mp['Version'] == project + "_" + version), 'Bugs'].values[0]
    data = data.append({"project" : area + "_" + project, "version" : version, "bugs": element}, ignore_index=True)


In [3]:
agg_funcs = {'bugs' : 'sum', 
         'version' : 'count'
}

data = data.groupby("project").agg(agg_funcs)
data


,bugs,version
project,,
Apache_CAMEL,1452.0,62
Apache_HBASE,858.0,70
Apache_HIVE,1219.0,21
Commons_CODEC,41.0,9
Commons_COLLECTIONS,93.0,7
Commons_COMPRESS,111.0,15
Commons_CONFIGURATION,135.0,11
Commons_CRYPTO,8.0,1
Commons_CSV,14.0,3


In [2]:
from pandas.errors import EmptyDataError

def evaluateForAlgo(base):
    data_map = dict()
    data_mrr = dict()
    data_hit1 = dict()
    data_hit5 = dict()
    data_hit10 = dict()
    
    for file in os.listdir(base):
        result = file.replace('smartshark_','').replace('_output.txt','').split("_")
        project = result[0] + "_" + result[1]
        area = result[0]
        version = "_".join(result[3:])
        if project not in data_map:
            data_map[project] = 0
            data_mrr[project] = 0
            data_hit1[project] = 0
            data_hit5[project] = 0
            data_hit10[project] = 0
        try:
            data_mp = pd.read_csv(base + "/" + file, header=None,delim_whitespace=True)  
            a_group = data_mp.groupby(0)
            for name, group in a_group:
                rr = 0;
                    
                summe = 0;
                retrieved_d = 0;   
                c = 0
                best_value = -1;
                for index, element in group[:10].iterrows():
                    if element[2] < best_value or best_value == -1:
                        best_value = element[2]
                    c += 1
                    retrieved_d += 1;
                    summe += retrieved_d /(element[2]+1)
                if best_value != -1:
                    data_mrr[project] += (1/ (best_value+1))
                data_map[project] += (summe/c)
                
                # These outputs are zero based!
                if best_value != -1 and best_value <= 0:
                    data_hit1[project] += 1
                if best_value != -1 and best_value <= 4:
                    data_hit5[project] += 1
                if best_value != -1 and best_value <= 9:
                    data_hit10[project] += 1
        except EmptyDataError:
             data_mp = pd.DataFrame()
    

    #data_mrr[project] = (1/ (data_mp[2]+1))    
    #print(data_mp)
    #c += 1
    #retrieved_d += 1;
    #summe += retrieved_d /(i+1)
    data['MAP_' + base] = pd.Series(data_map) /data["bugs"]
    data['MRR_' + base] =  pd.Series(data_mrr) / data["bugs"]
    data['HIT1_' + base] =  pd.Series(data_hit1) / data["bugs"]
    data['HIT5_' + base] =  pd.Series(data_hit5) / data["bugs"]
    data['HIT10_' + base] =  pd.Series(data_hit10) / data["bugs"]

In [5]:
evaluateForAlgo('amalgam')
evaluateForAlgo('amalgam_olddata')
evaluateForAlgo('amalgam_olddata_old')
data.to_csv('results_other_amalgam.csv')
data

,bugs,version,MAP_amalgam,MRR_amalgam,HIT1_amalgam,HIT5_amalgam,HIT10_amalgam,MAP_amalgam_olddata,MRR_amalgam_olddata,HIT1_amalgam_olddata,HIT5_amalgam_olddata,HIT10_amalgam_olddata,MAP_amalgam_olddata_old,MRR_amalgam_olddata_old,HIT1_amalgam_olddata_old,HIT5_amalgam_olddata_old,HIT10_amalgam_olddata_old
project,,,,,,,,,,,,,,,,,
Apache_CAMEL,1452.0,62,0.178198,0.213821,0.133609,0.292011,0.388430,0.035973,0.038934,0.011708,0.055096,0.080579,0.184478,0.217801,0.128788,0.305096,0.408402
Apache_HBASE,858.0,70,0.197020,0.247517,0.165501,0.336830,0.406760,0.014748,0.015688,0.003497,0.023310,0.032634,0.202759,0.245568,0.150350,0.350816,0.445221
Apache_HIVE,1219.0,21,0.155672,0.212731,0.143560,0.280558,0.342084,0.018748,0.022726,0.004922,0.038556,0.054143,0.169357,0.221179,0.144381,0.287941,0.363413
Commons_CODEC,41.0,9,0.644422,0.674042,0.536585,0.853659,0.926829,0.050860,0.049714,0.000000,0.048780,0.073171,0.581427,0.605827,0.414634,0.926829,0.975610
Commons_COLLECTIONS,93.0,7,0.322942,0.359486,0.247312,0.473118,0.548387,0.033933,0.034973,0.000000,0.064516,0.096774,0.320281,0.341222,0.215054,0.483871,0.569892
Commons_COMPRESS,111.0,15,0.335476,0.370961,0.216216,0.576577,0.756757,0.011736,0.009970,0.000000,0.000000,0.000000,0.353015,0.388213,0.207207,0.630631,0.819820
Commons_CONFIGURATION,135.0,11,0.347458,0.367266,0.281481,0.466667,0.555556,0.012540,0.012137,0.000000,0.000000,0.007407,0.366017,0.382189,0.266667,0.503704,0.622222
Commons_CRYPTO,8.0,1,0.157031,0.157031,0.125000,0.125000,0.375000,0.024597,0.024597,0.000000,0.000000,0.125000,0.183239,0.183239,0.125000,0.250000,0.375000
Commons_CSV,14.0,3,0.514263,0.568994,0.500000,0.714286,0.714286,0.065730,0.063050,0.000000,0.071429,0.142857,0.615476,0.645238,0.571429,0.857143,0.857143


In [6]:
data.mean(axis=0)

bugs                         213.215686
version                       13.941176
MAP_amalgam                    0.260076
MRR_amalgam                    0.297302
HIT1_amalgam                   0.207339
HIT5_amalgam                   0.393936
HIT10_amalgam                  0.474035
MAP_amalgam_olddata            0.029640
MRR_amalgam_olddata            0.031106
HIT1_amalgam_olddata           0.008673
HIT5_amalgam_olddata           0.035387
HIT10_amalgam_olddata          0.058719
MAP_amalgam_olddata_old        0.290111
MRR_amalgam_olddata_old        0.322305
HIT1_amalgam_olddata_old       0.212200
HIT5_amalgam_olddata_old       0.439415
HIT10_amalgam_olddata_old      0.552829
dtype: float64

In [17]:
import os
import pandas as pd

data_mp = pd.read_csv('../other/statistic.csv')
data_mp

,Project,Version,Bugs,Result
0,Apache_CAMEL,CAMEL_1_6_0,299,NaN
1,Apache_CAMEL,CAMEL_1_6_1,39,NaN
2,Apache_CAMEL,CAMEL_1_6_2,6,NaN
3,Apache_CAMEL,CAMEL_1_6_3,2,NaN
4,Apache_CAMEL,CAMEL_1_6_4,67,NaN
...,...,...,...,...
706,Wildfly_WFLY,WFLY_8_2_1,2,NaN
707,Wildfly_WFLY,WFLY_9_0_0,74,NaN
708,Wildfly_WFLY,WFLY_9_0_1,5,NaN
709,Wildfly_WFLY,WFLY_9_0_2,5,NaN


In [34]:
data = pd.DataFrame()
for file in os.listdir("../other/amalgam"):
    project = "_".join(file.replace('_output.txt','').split("_")[:2])
    version = "_".join(file.replace('_output.txt','').split("_")[2:])
    #print(project,version)
    element = data_mp.loc[(data_mp['Project'] == project) & (data_mp['Version'] == version), 'Bugs'].values[0]
    data = data.append({"project" : project, "version" : version, "bugs": element}, ignore_index=True)

In [35]:
agg_funcs = {'bugs' : 'sum', 
         'version' : 'count'
}

data = data.groupby("project").agg(agg_funcs)

data

,bugs,version
project,,
Apache_CAMEL,1452.0,62
Apache_HBASE,858.0,70
Apache_HIVE,1219.0,21
Commons_CODEC,41.0,9
Commons_COLLECTIONS,93.0,7
Commons_COMPRESS,111.0,15
Commons_CONFIGURATION,135.0,11
Commons_CRYPTO,8.0,1
Commons_CSV,14.0,3


In [1]:
from pandas.errors import EmptyDataError

def evaluateForAlgo(base, factor=0):
    data_map = dict()
    data_mrr = dict()
    data_hit1 = dict()
    data_hit5 = dict()
    data_hit10 = dict()
    
    for file in os.listdir('../other/' + base):
        print(file)
        project = "_".join(file.replace('_output.txt','').split("_")[:2])
        version = "_".join(file.replace('_output.txt','').split("_")[2:])
        if project not in data_map:
            data_map[project] = 0
            data_mrr[project] = 0
            data_hit1[project] = 0
            data_hit5[project] = 0
            data_hit10[project] = 0
        try:
            data_mp = pd.read_csv('../other/' + base + "/" + file, header=None,error_bad_lines=False,sep='\t')  
            a_group = data_mp.groupby(0)
            for name, group in a_group:
                rr = 0;
                    
                summe = 0;
                retrieved_d = 0;   
                c = 0
                best_value = -1;
                for index, element in group.iterrows():
                    if element[2] < best_value or best_value == -1:
                        best_value = element[2]
                    c += 1
                    retrieved_d += 1;
                    summe += retrieved_d /(element[2]+1)
                if best_value != -1:
                    data_mrr[project] += (1/ (best_value+1))
                data_map[project] += (summe/c)
                # These outputs are zero based!
                #print(best_value)
                if best_value != -1 and best_value == 0 + factor:
                    data_hit1[project] += 1
                if best_value != -1 and best_value <= 4 + factor:
                    data_hit5[project] += 1
                if best_value != -1 and best_value <= 9 + factor:
                    data_hit10[project] += 1
        except EmptyDataError:
             data_mp = pd.DataFrame()
    

    #data_mrr[project] = (1/ (data_mp[2]+1))    
    #print(data_mp)
    #c += 1
    #retrieved_d += 1;
    #summe += retrieved_d /(i+1)
    data['MAP_' + base] = pd.Series(data_map) /data["bugs"]
    data['MRR_' + base] =  pd.Series(data_mrr) / data["bugs"]
    data['HIT1_' + base] =  pd.Series(data_hit1) / data["bugs"]
    data['HIT5_' + base] =  pd.Series(data_hit5) / data["bugs"]
    data['HIT10_' + base] =  pd.Series(data_hit10) / data["bugs"]


evaluateForAlgo('buglocator')
evaluateForAlgo('bluir')
evaluateForAlgo('brtracer')
evaluateForAlgo('amalgam')
evaluateForAlgo('blia')
evaluateForAlgo('locus')
evaluateForAlgo('blizzard', 1)
data.to_csv('results_other_bench4bl.csv')
data

NameError: name 'os' is not defined